# 📊 Week 8 Live Demo: Gold-Layer Data Modeling & Dashboards
**Clogenai Academy – Data Pipeline Engineering Track**

## Session Overview
In this live demo, we'll:
1. Generate sample e-commerce data
2. Create Gold-layer dimension and fact tables
3. Write business analytics queries
4. Prepare data for dashboard visualizations

**Estimated Time**: 30 minutes


## 🔧 Setup: Catalog and Schemas


In [ ]:
%sql
-- Use academy catalog
USE CATALOG academy;

-- Create schemas
CREATE SCHEMA IF NOT EXISTS silver;
CREATE SCHEMA IF NOT EXISTS gold;


## 📦 Step 1: Generate Sample Data

We'll create sample e-commerce data for our demo:
- **Products**: 12 products across 3 categories
- **Orders**: 100 orders over 3 months (Jan-Mar 2024)


In [ ]:
%sql
-- Create sample products table
CREATE OR REPLACE TABLE academy.silver.products AS
SELECT 
    CONCAT('PROD', LPAD(CAST(id AS STRING), 3, '0')) AS product_id,
    CASE 
        WHEN id <= 3 THEN CONCAT('Laptop Model ', id)
        WHEN id <= 6 THEN CONCAT('Smartphone ', id - 3)
        WHEN id <= 9 THEN CONCAT('T-Shirt Style ', id - 6)
        ELSE CONCAT('Office Chair ', id - 9)
    END AS product_name,
    CASE 
        WHEN id <= 6 THEN 'Electronics'
        WHEN id <= 9 THEN 'Clothing'
        ELSE 'Home & Garden'
    END AS category,
    CASE 
        WHEN id <= 3 THEN 'Laptops'
        WHEN id <= 6 THEN 'Phones'
        WHEN id <= 9 THEN 'Apparel'
        ELSE 'Furniture'
    END AS subcategory,
    CASE 
        WHEN id <= 3 THEN 'TechBrand'
        WHEN id <= 6 THEN 'PhoneCo'
        WHEN id <= 9 THEN 'FashionHub'
        ELSE 'HomeComfort'
    END AS brand,
    CASE 
        WHEN id <= 3 THEN 800 + (id * 200)
        WHEN id <= 6 THEN 400 + ((id - 3) * 150)
        WHEN id <= 9 THEN 25 + ((id - 6) * 10)
        ELSE 150 + ((id - 9) * 50)
    END AS retail_price,
    CASE 
        WHEN id <= 3 THEN 600 + (id * 150)
        WHEN id <= 6 THEN 300 + ((id - 3) * 100)
        WHEN id <= 9 THEN 15 + ((id - 6) * 5)
        ELSE 100 + ((id - 9) * 30)
    END AS cost_price
FROM RANGE(1, 13); -- 12 products


In [ ]:
%sql
-- Verify products table
SELECT * FROM academy.silver.products ORDER BY product_id;


In [ ]:
%sql
-- Create sample orders table
CREATE OR REPLACE TABLE academy.silver.orders AS
SELECT 
    CONCAT('ORD', LPAD(CAST(id AS STRING), 4, '0')) AS order_id,
    CONCAT('CUST', LPAD(CAST(MOD(id * 7, 50) + 1 AS STRING), 3, '0')) AS customer_id,
    CONCAT('PROD', LPAD(CAST(MOD(id * 13, 12) + 1 AS STRING), 3, '0')) AS product_id,
    DATE_ADD('2024-01-01', CAST(MOD(id * 3, 90) AS INT)) AS order_date,
    TIMESTAMP(DATE_ADD('2024-01-01', CAST(MOD(id * 3, 90) AS INT))) AS order_timestamp,
    CAST(MOD(id * 5, 3) AS INT) + 1 AS quantity,
    CASE 
        WHEN MOD(id, 10) > 8 THEN 'Cancelled'
        WHEN MOD(id, 20) = 19 THEN 'Returned'
        ELSE 'Completed'
    END AS order_status
FROM RANGE(1, 101); -- 100 orders


In [ ]:
%sql
-- Verify orders table
SELECT * FROM academy.silver.orders ORDER BY order_date LIMIT 10;


## 🌟 Step 2: Create Gold-Layer Dimension Table

**Dimension Table**: `dim_product`
- Contains product attributes for analysis
- Includes calculated field: profit_margin


In [ ]:
%sql
-- Create product dimension table
CREATE OR REPLACE TABLE academy.gold.dim_product AS
SELECT 
    product_id,
    product_name,
    category,
    subcategory,
    brand,
    retail_price,
    cost_price,
    retail_price - cost_price AS profit_margin  -- Calculated field
FROM academy.silver.products;


In [ ]:
%sql
-- Verify dimension table
SELECT * FROM academy.gold.dim_product ORDER BY category, product_name;


## 📊 Step 3: Create Gold-Layer Fact Table

**Fact Table**: `fact_sales`
- **Grain**: One row per order (simplified for demo)
- **Metrics**: quantity, unit_price, line_total, profit
- **Business Rule**: Only include completed orders


In [ ]:
%sql
-- Create sales fact table
CREATE OR REPLACE TABLE academy.gold.fact_sales AS
SELECT 
    o.order_id,
    o.customer_id,
    o.product_id,
    o.order_date,
    o.quantity,
    p.retail_price AS unit_price,
    o.quantity * p.retail_price AS line_total,
    (o.quantity * p.retail_price) - (o.quantity * p.cost_price) AS profit
FROM academy.silver.orders o
JOIN academy.silver.products p ON o.product_id = p.product_id
WHERE o.order_status = 'Completed';  -- Business rule: only completed orders


In [ ]:
%sql
-- Verify fact table
SELECT COUNT(*) AS total_orders, 
       SUM(line_total) AS total_revenue,
       SUM(profit) AS total_profit
FROM academy.gold.fact_sales;


## 🔍 Step 4: Join Fact and Dimension

Demonstrate how easy it is to analyze data with star schema


In [ ]:
%sql
-- Example: Sales by product with details
SELECT 
    p.product_name,
    p.category,
    p.brand,
    COUNT(f.order_id) AS order_count,
    SUM(f.quantity) AS total_units_sold,
    SUM(f.line_total) AS total_revenue,
    SUM(f.profit) AS total_profit
FROM academy.gold.fact_sales f
JOIN academy.gold.dim_product p ON f.product_id = p.product_id
GROUP BY p.product_name, p.category, p.brand
ORDER BY total_revenue DESC
LIMIT 10;


## 📈 Step 5: Business Analytics Queries

These queries will power our dashboard visualizations


### Query 1: Sales KPIs
**Business Question**: "What are our key sales metrics?"


In [ ]:
%sql
-- Calculate key performance indicators
SELECT 
    COUNT(DISTINCT order_id) AS total_orders,
    SUM(line_total) AS total_revenue,
    AVG(line_total) AS avg_order_value,
    SUM(profit) AS total_profit,
    ROUND((SUM(profit) / SUM(line_total)) * 100, 2) AS profit_margin_pct
FROM academy.gold.fact_sales
WHERE order_date >= '2024-01-01';


### Query 2: Revenue by Category
**Business Question**: "Which product categories perform best?"


In [ ]:
%sql
-- Revenue analysis by category
SELECT 
    p.category,
    COUNT(DISTINCT f.order_id) AS order_count,
    SUM(f.line_total) AS total_revenue,
    ROUND(AVG(f.line_total), 2) AS avg_order_value,
    SUM(f.profit) AS total_profit
FROM academy.gold.fact_sales f
JOIN academy.gold.dim_product p ON f.product_id = p.product_id
GROUP BY p.category
ORDER BY total_revenue DESC;


### Query 3: Monthly Sales Trend
**Business Question**: "How are sales trending month-over-month?"


In [ ]:
%sql
-- Monthly trend with growth calculation
WITH monthly_sales AS (
    SELECT 
        DATE_TRUNC('month', order_date) AS month,
        COUNT(DISTINCT order_id) AS order_count,
        SUM(line_total) AS revenue,
        SUM(profit) AS profit
    FROM academy.gold.fact_sales
    GROUP BY DATE_TRUNC('month', order_date)
)
SELECT 
    month,
    order_count,
    revenue,
    profit,
    LAG(revenue) OVER (ORDER BY month) AS prev_month_revenue,
    revenue - LAG(revenue) OVER (ORDER BY month) AS revenue_change,
    ROUND(((revenue - LAG(revenue) OVER (ORDER BY month)) / 
     LAG(revenue) OVER (ORDER BY month) * 100), 2) AS growth_pct
FROM monthly_sales
ORDER BY month;


### Query 4: Top Products
**Business Question**: "What are our best-selling products?"


In [ ]:
%sql
-- Top 10 products by revenue
SELECT 
    p.product_name,
    p.category,
    p.brand,
    COUNT(f.order_id) AS order_count,
    SUM(f.quantity) AS units_sold,
    SUM(f.line_total) AS total_revenue,
    SUM(f.profit) AS total_profit,
    ROUND((SUM(f.profit) / SUM(f.line_total)) * 100, 2) AS profit_margin_pct
FROM academy.gold.fact_sales f
JOIN academy.gold.dim_product p ON f.product_id = p.product_id
GROUP BY p.product_name, p.category, p.brand
ORDER BY total_revenue DESC
LIMIT 10;


## 🎨 Step 6: Dashboard Preparation

**Next Steps** (Demonstrated in Databricks SQL):
1. Navigate to **Databricks SQL** → **Dashboards**
2. Create new dashboard: "Sales Performance Dashboard"
3. Add visualizations:
   - **Counter**: Total Revenue (from Query 1)
   - **Counter**: Total Orders (from Query 1)
   - **Bar Chart**: Revenue by Category (from Query 2)
   - **Line Chart**: Monthly Trend (from Query 3)
   - **Table**: Top Products (from Query 4)
4. Arrange layout and apply styling
5. Share with team


## ✅ Summary

**What We Built**:
- ✅ Sample e-commerce data (products + orders)
- ✅ Gold-layer dimension table (`dim_product`)
- ✅ Gold-layer fact table (`fact_sales`)
- ✅ 4 business analytics queries
- ✅ Data ready for dashboard visualizations

**Key Concepts**:
- **Star Schema**: Fact table surrounded by dimension tables
- **Grain**: One row per order (in our fact table)
- **Business Rules**: Only completed orders included
- **Calculated Fields**: Profit margin in dimension

**Next Steps**:
- Complete Week 8 assignment
- Build your own dashboard in Databricks SQL
- Explore optimization techniques (Z-ordering, partitioning)


## 🎓 Additional Practice

Try these exercises on your own:

1. **Create a customer dimension**: `gold.dim_customer`
2. **Add a date dimension**: `gold.dim_date`
3. **Write a query**: Revenue by brand
4. **Optimize**: Apply Z-ordering to `fact_sales`
5. **Create aggregation**: Daily sales summary table


---
**Clogenai Academy – Data Pipeline Engineering Track**  
**Week 8 Live Demo Notebook**  
**Version 1.0 – December 2025**
